In [1]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

In [2]:
train_raw = pd.read_csv('input/CSE7302c_train-1539668060821.csv', na_values='?')
train_raw.head()

,class,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1pt,jet1eta,jet1phi,jet1b.tag,...,jet4phi,jet4b.tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb,id
0,1,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343,1
1,1,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118,2
2,0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904,3
3,0,1.595839,-0.607811,0.007075,1.818450,-0.111906,0.847550,-0.566437,1.581239,2.173076,...,-1.274345,3.101961,0.823761,0.938191,0.971758,0.789176,0.430553,0.961357,0.957818,4
4,1,0.409391,-1.884684,-1.027292,1.672452,-1.604598,1.338015,0.055427,0.013466,2.173076,...,1.377130,3.101961,0.869418,1.222083,1.000627,0.545045,0.698653,0.977314,0.828786,5


In [3]:
test_raw = pd.read_csv('input/CSE7302c_test-1539668060821.csv', na_values='?')
test_raw.head()

,class,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1pt,jet1eta,jet1phi,jet1b.tag,...,jet4phi,jet4b.tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb,id
0,1,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487,68637
1,1,1.176566,0.104161,1.397002,0.479721,0.265513,1.135563,1.534831,-0.253291,0.000000,...,0.530334,0.000000,0.833175,0.773968,0.985750,1.103696,0.849140,0.937104,0.812364,68638
2,1,1.384098,0.116822,-1.179879,0.762913,-0.079782,1.019863,0.877318,1.276887,2.173076,...,0.504809,3.101961,0.959325,0.807376,1.191814,1.221210,0.861141,0.929341,0.838302,68639
3,1,1.330110,0.201557,1.173393,0.135309,-1.083159,0.728461,1.109032,-0.539903,1.086538,...,0.054775,3.101961,0.782073,0.672103,0.990002,0.734279,0.379074,0.764760,0.642924,68640
4,0,1.289848,-1.423023,-0.687162,0.131103,-1.135935,0.821350,0.296053,0.167582,2.173076,...,0.727328,0.000000,1.006643,0.867914,0.999090,1.109608,1.125094,0.882899,0.859396,68641


In [4]:
train_raw.nunique()

class                           2
lepton_pT                   18514
lepton_eta                   8919
lepton_phi                  11806
missing_energy_magnitude    66854
missing_energy_phi          66804
jet1pt                      25954
jet1eta                      9422
jet1phi                     11713
jet1b.tag                       5
jet2pt                      22270
jet2eta                      9658
jet2phi                     11827
jet2b.tag                       5
jet3pt                      17882
jet3eta                     10089
jet3phi                     11753
jet3b.tag                       5
jet4pt                      13935
jet4eta                     10516
jet4phi                     11801
jet4b.tag                       5
m_jj                        65748
m_jjj                       57372
m_lv                        42247
m_jlv                       60935
m_bb                        62851
m_wbb                       63301
m_wwbb                      64483
id            

In [5]:
test_raw.nunique()

class                           2
lepton_pT                   13786
lepton_eta                   7895
lepton_phi                  10476
missing_energy_magnitude    29132
missing_energy_phi          29138
jet1pt                      17678
jet1eta                      7838
jet1phi                     10383
jet1b.tag                       5
jet2pt                      15784
jet2eta                      8145
jet2phi                     10507
jet2b.tag                       5
jet3pt                      13492
jet3eta                      8498
jet3phi                     10487
jet3b.tag                       5
jet4pt                      10920
jet4eta                      8978
jet4phi                      6233
jet4b.tag                       3
m_jj                        28628
m_jjj                       26074
m_lv                        20085
m_jlv                       27166
m_bb                        27697
m_wbb                       27977
m_wwbb                      28235
id            

In [6]:
## We have few columns that have unique values of less than 5, these are good candidates for being categorical.

In [7]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68636 entries, 0 to 68635
Data columns (total 30 columns):
class                       68636 non-null int64
lepton_pT                   68636 non-null float64
lepton_eta                  68636 non-null float64
lepton_phi                  68636 non-null float64
missing_energy_magnitude    68636 non-null float64
missing_energy_phi          68636 non-null float64
jet1pt                      68636 non-null float64
jet1eta                     68636 non-null float64
jet1phi                     68636 non-null float64
jet1b.tag                   68636 non-null float64
jet2pt                      68636 non-null float64
jet2eta                     68636 non-null float64
jet2phi                     68636 non-null float64
jet2b.tag                   68636 non-null float64
jet3pt                      68636 non-null float64
jet3eta                     68636 non-null float64
jet3phi                     68636 non-null float64
jet3b.tag                 

In [8]:
test_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29414 entries, 0 to 29413
Data columns (total 30 columns):
class                       29414 non-null int64
lepton_pT                   29414 non-null float64
lepton_eta                  29414 non-null float64
lepton_phi                  29414 non-null float64
missing_energy_magnitude    29414 non-null float64
missing_energy_phi          29414 non-null float64
jet1pt                      29414 non-null float64
jet1eta                     29414 non-null float64
jet1phi                     29414 non-null float64
jet1b.tag                   29414 non-null float64
jet2pt                      29414 non-null float64
jet2eta                     29414 non-null float64
jet2phi                     29414 non-null float64
jet2b.tag                   29414 non-null float64
jet3pt                      29414 non-null float64
jet3eta                     29414 non-null float64
jet3phi                     29414 non-null float64
jet3b.tag                 

In [9]:
## we know that 
# train size:  68636  observations  30 attributes
# test size:   29414 observations 30 attributes

# We have 68636 and 29414 unique ids in train and test respectively and this nominal data can be removed.
train_data = train_raw.drop('id', axis=1)
test_data = test_raw.drop('id', axis=1)
train_data.head()

,class,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1pt,jet1eta,jet1phi,jet1b.tag,...,jet4eta,jet4phi,jet4b.tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
1,1,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
2,0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
3,0,1.595839,-0.607811,0.007075,1.818450,-0.111906,0.847550,-0.566437,1.581239,2.173076,...,-0.654227,-1.274345,3.101961,0.823761,0.938191,0.971758,0.789176,0.430553,0.961357,0.957818
4,1,0.409391,-1.884684,-1.027292,1.672452,-1.604598,1.338015,0.055427,0.013466,2.173076,...,0.069496,1.377130,3.101961,0.869418,1.222083,1.000627,0.545045,0.698653,0.977314,0.828786


In [10]:
test_data.head()

,class,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1pt,jet1eta,jet1phi,jet1b.tag,...,jet4eta,jet4phi,jet4b.tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487
1,1,1.176566,0.104161,1.397002,0.479721,0.265513,1.135563,1.534831,-0.253291,0.000000,...,0.268541,0.530334,0.000000,0.833175,0.773968,0.985750,1.103696,0.849140,0.937104,0.812364
2,1,1.384098,0.116822,-1.179879,0.762913,-0.079782,1.019863,0.877318,1.276887,2.173076,...,0.846521,0.504809,3.101961,0.959325,0.807376,1.191814,1.221210,0.861141,0.929341,0.838302
3,1,1.330110,0.201557,1.173393,0.135309,-1.083159,0.728461,1.109032,-0.539903,1.086538,...,1.007255,0.054775,3.101961,0.782073,0.672103,0.990002,0.734279,0.379074,0.764760,0.642924
4,0,1.289848,-1.423023,-0.687162,0.131103,-1.135935,0.821350,0.296053,0.167582,2.173076,...,1.105529,0.727328,0.000000,1.006643,0.867914,0.999090,1.109608,1.125094,0.882899,0.859396


In [11]:
cat_cols = train_data.columns[train_data.nunique() <= 5]
print(cat_cols)
num_cols = train_data.columns[train_data.nunique() > 5]
num_cols

Index(['class', 'jet1b.tag', 'jet2b.tag', 'jet3b.tag', 'jet4b.tag'], dtype='object')


Index(['lepton_pT', 'lepton_eta', 'lepton_phi', 'missing_energy_magnitude',
       'missing_energy_phi', 'jet1pt', 'jet1eta', 'jet1phi', 'jet2pt',
       'jet2eta', 'jet2phi', 'jet3pt', 'jet3eta', 'jet3phi', 'jet4pt',
       'jet4eta', 'jet4phi', 'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb',
       'm_wwbb'],
      dtype='object')

In [12]:
## type casting
train_data[cat_cols] = train_data[cat_cols].astype('category')
test_data[cat_cols] = test_data[cat_cols].astype('category')

In [13]:
train_data.dtypes

class                       category
lepton_pT                    float64
lepton_eta                   float64
lepton_phi                   float64
missing_energy_magnitude     float64
missing_energy_phi           float64
jet1pt                       float64
jet1eta                      float64
jet1phi                      float64
jet1b.tag                   category
jet2pt                       float64
jet2eta                      float64
jet2phi                      float64
jet2b.tag                   category
jet3pt                       float64
jet3eta                      float64
jet3phi                      float64
jet3b.tag                   category
jet4pt                       float64
jet4eta                      float64
jet4phi                      float64
jet4b.tag                   category
m_jj                         float64
m_jjj                        float64
m_lv                         float64
m_jlv                        float64
m_bb                         float64
m

In [14]:
test_data.dtypes

class                       category
lepton_pT                    float64
lepton_eta                   float64
lepton_phi                   float64
missing_energy_magnitude     float64
missing_energy_phi           float64
jet1pt                       float64
jet1eta                      float64
jet1phi                      float64
jet1b.tag                   category
jet2pt                       float64
jet2eta                      float64
jet2phi                      float64
jet2b.tag                   category
jet3pt                       float64
jet3eta                      float64
jet3phi                      float64
jet3b.tag                   category
jet4pt                       float64
jet4eta                      float64
jet4phi                      float64
jet4b.tag                   category
m_jj                         float64
m_jjj                        float64
m_lv                         float64
m_jlv                        float64
m_bb                         float64
m

In [15]:
## NA Values
train_data.isna().sum()

class                       0
lepton_pT                   0
lepton_eta                  0
lepton_phi                  0
missing_energy_magnitude    0
missing_energy_phi          0
jet1pt                      0
jet1eta                     0
jet1phi                     0
jet1b.tag                   0
jet2pt                      0
jet2eta                     0
jet2phi                     0
jet2b.tag                   0
jet3pt                      0
jet3eta                     0
jet3phi                     0
jet3b.tag                   0
jet4pt                      0
jet4eta                     0
jet4phi                     0
jet4b.tag                   0
m_jj                        0
m_jjj                       0
m_lv                        0
m_jlv                       0
m_bb                        0
m_wbb                       0
m_wwbb                      0
dtype: int64

In [16]:
test_data.isna().sum()

class                       0
lepton_pT                   0
lepton_eta                  0
lepton_phi                  0
missing_energy_magnitude    0
missing_energy_phi          0
jet1pt                      0
jet1eta                     0
jet1phi                     0
jet1b.tag                   0
jet2pt                      0
jet2eta                     0
jet2phi                     0
jet2b.tag                   0
jet3pt                      0
jet3eta                     0
jet3phi                     0
jet3b.tag                   0
jet4pt                      0
jet4eta                     0
jet4phi                     1
jet4b.tag                   1
m_jj                        1
m_jjj                       1
m_lv                        1
m_jlv                       1
m_bb                        1
m_wbb                       1
m_wwbb                      1
dtype: int64

In [17]:
## So we have NA values in test but not in train. Good to know.

In [18]:
## Ref: https://scikit-learn.org/stable/modules/impute.html#impute
## categorical data
cat_imputer = SimpleImputer(strategy='most_frequent')
test_data[cat_cols] = pd.DataFrame(cat_imputer.fit_transform(test_data[cat_cols]), columns=list(cat_cols))

In [19]:
test_data.isna().sum()

class                       0
lepton_pT                   0
lepton_eta                  0
lepton_phi                  0
missing_energy_magnitude    0
missing_energy_phi          0
jet1pt                      0
jet1eta                     0
jet1phi                     0
jet1b.tag                   0
jet2pt                      0
jet2eta                     0
jet2phi                     0
jet2b.tag                   0
jet3pt                      0
jet3eta                     0
jet3phi                     0
jet3b.tag                   0
jet4pt                      0
jet4eta                     0
jet4phi                     1
jet4b.tag                   0
m_jj                        1
m_jjj                       1
m_lv                        1
m_jlv                       1
m_bb                        1
m_wbb                       1
m_wwbb                      1
dtype: int64

In [20]:
## Ref: https://scikit-learn.org/stable/modules/impute.html#knnimpute
## impute numerical data
num_imputer = KNNImputer()
test_data[num_cols] = pd.DataFrame(num_imputer.fit_transform(test_data[num_cols]), columns=num_cols)

In [22]:
test_data.isna().sum()

class                       0
lepton_pT                   0
lepton_eta                  0
lepton_phi                  0
missing_energy_magnitude    0
missing_energy_phi          0
jet1pt                      0
jet1eta                     0
jet1phi                     0
jet1b.tag                   0
jet2pt                      0
jet2eta                     0
jet2phi                     0
jet2b.tag                   0
jet3pt                      0
jet3eta                     0
jet3phi                     0
jet3b.tag                   0
jet4pt                      0
jet4eta                     0
jet4phi                     0
jet4b.tag                   0
m_jj                        0
m_jjj                       0
m_lv                        0
m_jlv                       0
m_bb                        0
m_wbb                       0
m_wwbb                      0
dtype: int64

In [21]:
test_data.dtypes

class                        object
lepton_pT                   float64
lepton_eta                  float64
lepton_phi                  float64
missing_energy_magnitude    float64
missing_energy_phi          float64
jet1pt                      float64
jet1eta                     float64
jet1phi                     float64
jet1b.tag                    object
jet2pt                      float64
jet2eta                     float64
jet2phi                     float64
jet2b.tag                    object
jet3pt                      float64
jet3eta                     float64
jet3phi                     float64
jet3b.tag                    object
jet4pt                      float64
jet4eta                     float64
jet4phi                     float64
jet4b.tag                    object
m_jj                        float64
m_jjj                       float64
m_lv                        float64
m_jlv                       float64
m_bb                        float64
m_wbb                       

In [23]:
## Let's drop 'class' from cat_cols since it's target variable
## Lets also drop 'jet4b.tag' for now since there's class imbalance in the variables
cat_cols = cat_cols.drop('class')
cat_cols = list(cat_cols)
cat_cols

['jet1b.tag', 'jet2b.tag', 'jet3b.tag', 'jet4b.tag']

In [24]:
x_train, y_train = train_data.drop('class', axis=1), train_data['class'].astype('category')
x_test, y_test = test_data.drop('class', axis=1), test_data['class'].astype('category')
print(x_train.shape)
print(x_test.shape)

(68636, 28)
(29414, 28)


In [25]:
from sklearn.preprocessing import OneHotEncoder
## LabelEncoder
onehotencoder = OneHotEncoder(handle_unknown='ignore')

## Fit method
onehotencoder = onehotencoder.fit(train_data[cat_cols])

## Get names for new columns
ohe_cat_col_names = onehotencoder.get_feature_names(cat_cols)

## Print the above columns names
ohe_cat_col_names

array(['jet1b.tag_0.0', 'jet1b.tag_1.086538',
       'jet1b.tag_1.0865380759999999', 'jet1b.tag_2.173076',
       'jet1b.tag_2.1730761530000002', 'jet2b.tag_0.0',
       'jet2b.tag_1.107436', 'jet2b.tag_1.107436061',
       'jet2b.tag_2.214872', 'jet2b.tag_2.214872122', 'jet3b.tag_0.0',
       'jet3b.tag_1.2741120000000001', 'jet3b.tag_1.274112225',
       'jet3b.tag_2.5482240000000003', 'jet3b.tag_2.548224449',
       'jet4b.tag_0.0', 'jet4b.tag_1.5509806869999998',
       'jet4b.tag_1.550981', 'jet4b.tag_3.101961',
       'jet4b.tag_3.1019613739999996'], dtype=object)

In [26]:
## Encode x_train category columns 
X_train_cat_onehotencoded = onehotencoder.transform(x_train[cat_cols]).toarray()
X_train_cat_onehotencoded = pd.DataFrame(X_train_cat_onehotencoded, columns=ohe_cat_col_names)
X_train_cat_onehotencoded.head()

,jet1b.tag_0.0,jet1b.tag_1.086538,jet1b.tag_1.0865380759999999,jet1b.tag_2.173076,jet1b.tag_2.1730761530000002,jet2b.tag_0.0,jet2b.tag_1.107436,jet2b.tag_1.107436061,jet2b.tag_2.214872,jet2b.tag_2.214872122,jet3b.tag_0.0,jet3b.tag_1.2741120000000001,jet3b.tag_1.274112225,jet3b.tag_2.5482240000000003,jet3b.tag_2.548224449,jet4b.tag_0.0,jet4b.tag_1.5509806869999998,jet4b.tag_1.550981,jet4b.tag_3.101961,jet4b.tag_3.1019613739999996
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [27]:
## Encode x_test category columns 
X_test_cat_onehotencoded = onehotencoder.transform(x_test[cat_cols]).toarray()
X_test_cat_onehotencoded = pd.DataFrame(X_test_cat_onehotencoded, columns=ohe_cat_col_names)
X_test_cat_onehotencoded.head()

,jet1b.tag_0.0,jet1b.tag_1.086538,jet1b.tag_1.0865380759999999,jet1b.tag_2.173076,jet1b.tag_2.1730761530000002,jet2b.tag_0.0,jet2b.tag_1.107436,jet2b.tag_1.107436061,jet2b.tag_2.214872,jet2b.tag_2.214872122,jet3b.tag_0.0,jet3b.tag_1.2741120000000001,jet3b.tag_1.274112225,jet3b.tag_2.5482240000000003,jet3b.tag_2.548224449,jet4b.tag_0.0,jet4b.tag_1.5509806869999998,jet4b.tag_1.550981,jet4b.tag_3.101961,jet4b.tag_3.1019613739999996
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [28]:
x_train = pd.concat([X_train_cat_onehotencoded, x_train[num_cols]], axis=1)
x_train.shape

(68636, 44)

In [29]:
x_test = pd.concat([X_test_cat_onehotencoded, x_test[num_cols]], axis=1)
x_test.shape

(29414, 44)

In [30]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)
y_train_pred = log_reg.predict(x_train)
print(f'train accuracy: {accuracy_score(y_train, y_train_pred)}')
y_test_pred = log_reg.predict(x_test)
print(f'test accuracy: {accuracy_score(y_test, y_test_pred)}')

train accuracy: 0.641762340462731
test accuracy: 0.6412252668797171
